## LGMVIP - Advance Level Task - *Next Word Predictor*
### *Vinay Reddy*


Importing Libraries

In [2]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

Uploading Dataset from system


In [3]:
from google.colab import files
uploaded = files.upload()

Saving text.txt to text.txt


Loading the data

In [4]:
text= open('text.txt',encoding='UTF-8').read().lower()
print('corpus length:', len(text))

corpus length: 582453


Splitting dataset into one word each

In [5]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

Making list of sorted unique words

In [6]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

Feature Engineering and One Hot Encoding

In [7]:
WORD_LENGTH = 5
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['skip', 'to', 'content', 'search', 'or']
jump


Storing features and corresponding labels

In [8]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [9]:
print(X[0][0])

[False False False ... False False False]


Building the model

In [10]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))


Training our model

In [13]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/2
812/812 [==============================] - 274s 335ms/step - loss: 6.0168 - accuracy: 0.1068 - val_loss: 7.1662 - val_accuracy: 0.0988
Epoch 2/2
812/812 [==============================] - 258s 318ms/step - loss: 5.7834 - accuracy: 0.1464 - val_loss: 7.6780 - val_accuracy: 0.1059


Loading our Model

In [14]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

Making Predictions

In [15]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("How are you ".lower())

how
are
you


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [16]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

Function for Prediction

In [17]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

Result

In [26]:
q =  "The giant ball is in the bag "
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:4])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  The giant ball is in the bag 
Sequence:  the giant ball is
the
giant
ball
is
next possible words:  ['to', 'that', 'in', 'and', 'a']


Thank You

---

